In [3]:
!pip install geopandas

   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ------- -------------------------------- 2.9/16.2 MB 15.2 MB/s eta 0:00:01
   ---------------- ----------------------- 6.6/16.2 MB 16.1 MB/s eta 0:00:01
   ------------------------- -------------- 10.2/16.2 MB 16.8 MB/s eta 0:00:01
   ------------------------------- -------- 12.8/16.2 MB 15.2 MB/s eta 0:00:01
   -------------------------------------- - 15.5/16.2 MB 14.7 MB/s eta 0:00:01
   ---------------------------------------- 16.2/16.2 MB 13.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   --------------- ------------------------ 2.4/6.2 MB 11.2 MB/s eta 0:00:01
   ----------------------------------- ---- 5.5/6.2 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 6.2/6.2 MB 12.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 15.1 MB/s eta 0:00:00


In [5]:
!pip install mercantile

In [6]:
import pandas as pd
import geopandas as gpd
from shapely import geometry
import mercantile
from tqdm import tqdm
import os
import tempfile

In [7]:
# Geometry copied from https://geojson.io
aoi_geom = {
    "coordinates": [
          [
            [
              68.47164779002796,
              34.9546713429649
            ],
            [
              68.47164779002796,
              34.161837779504765
            ],
            [
              69.9151307000588,
              34.161837779504765
            ],
            [
              69.9151307000588,
              34.9546713429649
            ],
            [
              68.47164779002796,
              34.9546713429649
            ]
          ]
        ],
    "type": "Polygon",
}
aoi_shape = geometry.shape(aoi_geom)
minx, miny, maxx, maxy = aoi_shape.bounds

output_fn = "example_building_footprints_AFG.geojson"

In [8]:
quad_keys = set()
for tile in list(mercantile.tiles(minx, miny, maxx, maxy, zooms=9)):
    quad_keys.add(mercantile.quadkey(tile))
quad_keys = list(quad_keys)
print(f"The input area spans {len(quad_keys)} tiles: {quad_keys}")

The input area spans 9 tiles: ['123102021', '123102032', '123102201', '123102030', '123102033', '123102023', '123102211', '123102031', '123102210']


In [12]:
df = pd.read_csv(
    "https://minedbuildings.z5.web.core.windows.net/global-buildings/dataset-links.csv", dtype=str
)
df.head()

,Location,QuadKey,Url,Size,UploadDate
0,Abyei,122320113,https://minedbuildings.z5.web.core.windows.net...,74.7KB,2025-01-06
1,Abyei,122320131,https://minedbuildings.z5.web.core.windows.net...,8.3KB,2025-01-06
2,Abyei,122321002,https://minedbuildings.z5.web.core.windows.net...,392.3KB,2025-01-06
3,Abyei,122321003,https://minedbuildings.z5.web.core.windows.net...,72.7KB,2025-01-06
4,Abyei,122321020,https://minedbuildings.z5.web.core.windows.net...,1.2MB,2025-01-06


In [14]:
combined_gdf = combined_gdf.to_crs('EPSG:4326')
combined_gdf.to_file(output_fn, driver="GeoJSON")
print(f"Saved GeoJSON file to: {os.path.abspath(output_fn)}")

AttributeError: 'Series' object has no attribute 'to_crs'

In [5]:

# Assuming df is your DataFrame containing 'QuadKey' and 'Url' columns, and quad_keys is a list of QuadKeys to process
# Assuming aoi_shape is your Area of Interest (AOI) shape for filtering geometries

idx = 0
combined_gdf = gpd.GeoDataFrame()

for quad_key in tqdm(quad_keys):
    rows = df[df["QuadKey"] == quad_key]
    if rows.shape[0] >= 1:  # Allow multiple rows for the same QuadKey
        tmp_gdf_list = []
        for _, row in rows.iterrows():
            url = row["Url"]
            df2 = pd.read_json(url, lines=True)
            df2["geometry"] = df2["geometry"].apply(geometry.shape)
            gdf = gpd.GeoDataFrame(df2, crs=4326)
            tmp_gdf_list.append(gdf)

        # Combine all GeoDataFrames for the current QuadKey
        combined_gdf_quad_key = pd.concat(tmp_gdf_list, ignore_index=True)

        # Filter geometries within the AOI
        combined_gdf_quad_key = combined_gdf_quad_key[combined_gdf_quad_key.geometry.within(aoi_shape)]

        # Update 'id' based on idx
        combined_gdf_quad_key['id'] = range(idx, idx + len(combined_gdf_quad_key))
        idx += len(combined_gdf_quad_key)

        # Combine with the main GeoDataFrame
        combined_gdf = pd.concat([combined_gdf, combined_gdf_quad_key], ignore_index=True)
    else:
        raise ValueError(f"No rows found for QuadKey: {quad_key}")




100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [02:28<00:00, 16.46s/it]


In [6]:
# Save the combined GeoDataFrame to a GeoJSON file
combined_gdf.to_file(output_fn, driver="GeoJSON")
print(f"Saved GeoJSON file to: {os.path.abspath(output_fn)}")

Saved GeoJSON file to: C:\Users\ahmedshehata\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\example_building_footprints_AFG.geojson
